In [1]:
# Note that before running this peace of code you must install the microsoft c++ compiler (MSVC)

In [ ]:
# These libraris are used for choosing the wanted device as our mic & adjustement of the volume
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
#-----------------------------------------------------------------
# Used for Delay and system commands
import sys
import time
#-----------------------------------------------------------------
# Used to detect voice activity detection
import numpy as np
import webrtcvad
#-----------------------------------------------------------------
# Used for streaming
import sounddevice as sd
#-----------------------------------------------------------------


channels = [1]

mapping  = [c - 1 for c in channels]

device_info = sd.query_devices(None, 'input')
sample_rate = 16000

frame_duration = 30
downsample = 1

frame = sample_rate * frame_duration / 1000 


vad = webrtcvad.Vad(2) # The number used in between the brackets is the sensitivity level

print("reading audio stream from default audio input device:\n" + str(sd.query_devices()) + '\n')
print(F"audio input channels to process: {channels}")
print(F"sample_rate: {sample_rate}")
print(F"window size: {frame_duration} ms")
print(F"datums per window: {frame}")

def voice_activity_detection(audio_data):
    return vad.is_speech(audio_data, sample_rate)



def audio_callback(indata, frames, time, status):
    """This is called (from a separate thread) for each audio block."""
    if status:
        
        print(F"underlying audio stack warning:{status}", file=sys.stderr)

    assert frames == frame
    audio_data = indata[::downsample, mapping]
    audio_data = map(lambda x: (x+1)/2, audio_data)
    audio_data = np.fromiter(audio_data, np.float16)

    audio_data = audio_data.tobytes()
    global detection
    detection = voice_activity_detection(audio_data)
    print(f'{detection} \r', end="")


currentVolumeDb = volume.GetMasterVolumeLevel()



b = 0

while True:
    
    
        

    for i in range (0, 2, 1):
        
        with sd.InputStream(
        device=None, 
        channels=max(channels),
        samplerate=sample_rate,
        blocksize=int(frame),
        callback=audio_callback):
            
            time.sleep(0.1)
            
    while True:
        
        # These are the conditions for adjusting the sound while someone is speaking or while someone isn't
        
        if detection == False :
            
            b+=1
            
            if b >= (10) : 
                
                
                b=0
                
                v2 = volume.GetMasterVolumeLevel()
                
                if v2 == -22 :
                
                    volume.SetMasterVolumeLevel(currentVolumeDb, None)
                else:
                    volume.SetMasterVolumeLevel(v2, None)
                
                currentVolumeDb = volume.GetMasterVolumeLevel()                
                
        else:
            
            b=0
            
            volume.SetMasterVolumeLevel(-22, None)
        
        print(f'{detection} \r', end="")
        
        break